In [ ]:
import pandas as pd
import numpy as np
import re
import os
import torch
import pickle
import sklearn
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoModel,AutoTokenizer
from nltk.corpus import stopwords
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from nltk.stem.snowball import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModel, BertModel, AutoTokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import nltk
nltk.download('stopwords')
nltk.download('punkt')

## Some funcs

In [ ]:
stemmer = SnowballStemmer('russian')

stop_words = stopwords.words('english')

def doc_prepr(df, column_name='text'):
    new_df = df

    for i, text in tqdm(enumerate(df[column_name])):
        if (len(text) < 10):
            text = ''
        text = re.sub(r'[\n\t]+(\d){1,3}(?=[\n\t])', '', text)
        text = re.sub(r'(?<!\.)( *(\n|\t) *)(?=[A-ZРђ-РЇРЃ])', '.\g<1>', text)
        text = re.sub(r'(?<!\.)( *(\n|\t) *)(\d{1,3} *([\.\)/][\.\)\d/]*)* *)(?=[A-ZРђ-РЇРЃ])', '.\g<1>\g<3>', text)
        text = re.sub(r'(?<!\.)( *(\n|\t) *)([A-Za-zРђ-РЇРЃР°-СЏС‘][\.\)] *)(?=[A-ZРђ-РЇРЃ])', '.\g<1>\g<3>', text)
        text = re.sub(r'((\t)|(\n))ERROR:#.{,10}?!', '', text)
        text = re.sub(r'\.+', '.', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'_+', ' ', text)

        new_df.loc[i, 'text'] = text

    return new_df


def preprocessing(text, tokenizer, stop_words, del_sw=False, lemm=False, stem=True):
        text = str(text)
        text = text.lower()
        text = re.sub(r'([^\s\w])', ' ', text)
        text = re.sub(r'\d+', ' ', text)
        text = re.sub(r'_', ' ', text)
        text = re.sub(r'№', 'номер ', text)
        text = re.sub(r"\s+", ' ', text)
        text = re.sub(r' \.', '.', text)
        text = re.sub(r'\.([^\s])', '. \g<1>', text)
        text = re.sub(r"^\s$", '', text)
        # text = remove_fio(text)
        text = tokenizer(text)
#         text = tokenizer(text, padding=True, truncation=True, max_length=64, return_tensors='pt')
        if del_sw:
                text = del_stopwords(text, stop_words)
        if lemm:
                text = lemmatize(text)
        if stem:
                text = stemm(text)

        return text


def detokenization(list_token_docs):

    detoken_docs = []
    for token_docs in list_token_docs:
            word =' '.join(token_docs)
            detoken_docs.append(word)
    return detoken_docs


def remove_fio(text):
    text = str(text)
    text = re.sub(r'[Рђ-РЇРЃ]\.\s*[Рђ-РЇРЃ]\.\s*[Рђ-РЇРЃ][Рђ-РЇР°-СЏРЃС‘]+', '', text)
    text = re.sub(r'[Рђ-РЇРЃ][Рђ-РЇР°-СЏРЃС‘]+?\s*[Рђ-РЇРЃ]\.\s*[Рђ-РЇРЃ]([\.\s]|$)', '', text)
    text = re.sub(r'[Рђ-РЇ][Р°-СЏС‘]+\s*[Рђ-РЇ][Р°-СЏС‘]+\s*[Рђ-РЇ][Р°-СЏС‘]+', ' ', text)
    text = re.sub(
        r'[Р°-СЏС‘]+?((РѕРІ)|(РµРІ)|(С‘РІ)|(РёРЅ)|(СЏРЅ)|(РµРЅРєРѕ)|(СѓРє)|(СЋРє)|(Р°Р»Рѕ)|(РµР»Рѕ)|(РґР·Рµ)|(РёР»Рё)|(РєРѕ)|(РІРёС‡)|(РёС…)|(СЃРєРёР№)|(СЃРєР°СЏ))(Р°)?\s+[Р°-СЏС‘]+?\s+[Р°-СЏС‘]+?((РІРёС‡)|(РІРЅР°))',
        ' ', text, flags=re.I)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text


def del_stopwords(word_list, stop_words):
        new_word_list = [word for word in word_list if word not in stop_words and len(word) > 2]
        return new_word_list


def stemm(word_list):
        stem_word_list = [stemmer.stem(word) for word in word_list]
        return stem_word_list


def lemmatize(text_data):
        morph = pymorphy2.MorphAnalyzer()
        lemm_text_data = [morph.parse(word)[0].normal_form for word in text_data]
        return lemm_text_data


def sentenize(df, column_name='text', first_tokens_num=2000):
    for i, doc in enumerate(df[column_name]):
        df.loc[i, column_name] = ' '.join(doc.split()[:first_tokens_num])
        fnames, sents = [], []

    for i, doc_name in tqdm(enumerate(df['fname'])):
        cur_sents = [j.text for j in list(razdel.sentenize(df.loc[i, 'text'])) if bool(re.search(r'[Р°-СЏС‘]{3,}', j.text, re.I))]
        fnames.extend([doc_name] * len(cur_sents))
        sents.extend(cur_sents)

    new_df = pd.DataFrame({'fname': fnames, 'sent': sents})

    return new_df

def labse_vectorize(sents, model_tokenizer, model_vectorizer, batch_size=100, cuda=False):
    if cuda:
        device = torch.device('cuda', 0)
        model_vectorizer = model_vectorizer.to(device)
    else:
        device = 'cpu'
    labse_vectors = np.zeros((len(sents), 768))
    batch_count = len(sents)//batch_size + int(bool(len(sents)%batch_size))
    idx = 0
    for i in tqdm(range(batch_count)):
        cur_sents = sents[i*batch_size:i*batch_size+batch_size]
        cur_sents = list(map(str,cur_sents))
        with torch.no_grad():
            tokenized_sent = model_tokenizer(cur_sents, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
            sent_embedding = model_vectorizer(**tokenized_sent).pooler_output.to('cpu')
        labse_vectors[i*batch_size:i*batch_size+batch_size,:] = sent_embedding.detach().numpy()
        with torch.no_grad():
            del(tokenized_sent)
            del(sent_embedding)
        torch.cuda.empty_cache()

    return labse_vectors


def my_cross_validate(df, ml_model, kfold_model, vectors, target_col_name, save_test=True, print_res=True, oversampling=False, undersampling=False, random_state=1):

    X = vectors
    y = df[target_col_name]

    if oversampling:
        smote_dict = {i:y.value_counts()[i]*10 for i in np.unique(y) if i!=0}
        smote = SMOTE(sampling_strategy=smote_dict, random_state=1, k_neighbors=5, n_jobs=1)
        X, y = smote.fit_resample(X, y)

    indices = kfold_model.split(X,y)

    clf = ml_model
    y_preds = []
    classification_reports = []

    precision_scores = []
    recall_scores = []
    f1_scores = []
    test_dfs = []

    if len(y.unique()) > 2:
        average_param = 'macro'
        classification_report_ = True
    else:
        average_param = 'binary'
        classification_report_ = False

    for i, (train_index, test_index) in tqdm(enumerate(indices)):
        print(f'Fold №: {i}')

        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]

        if (undersampling):
            und_samp_dict = {i:int(y_train.value_counts()[i]//10) for i in np.unique(y) if i==0}
            und_sampler = RandomUnderSampler(sampling_strategy=und_samp_dict,random_state=42)
            X_train, y_train = und_sampler.fit_resample(X_train, y_train)

        print(y_train.value_counts())

        clf.fit(X_train, y_train)
        y_preds.append(clf.predict(X_test))

        if (save_test):
            temp_df = df.copy()
            temp_df = temp_df.loc[test_index]
            temp_df['pred_tag'] = y_preds[i]
            test_dfs.append(temp_df)

        labels = y_test

        if classification_report_:
            classification_reports.append(classification_report(labels, y_preds[i], output_dict=True))

        prec_score = precision_score(labels, y_preds[i], average=average_param)
        rec_score = recall_score(labels, y_preds[i], average=average_param)
        f1_scor = f1_score(labels, y_preds[i], average=average_param)
        precision_scores.append(prec_score)
        recall_scores.append(rec_score)
        f1_scores.append(f1_scor)

        if (print_res==False):
            return

        print(f'Precision: {prec_score}')
        print(f'Recall: {rec_score}')
        print(f'F1: {f1_scor}')
        print()

    res_avg_metrics = pd.DataFrame({'model': [str(clf).split('(')[0]], 'avg_precision': np.array(precision_scores).mean(), 'avg_recall': np.array(recall_scores).mean(), 'avg_f1': np.array(f1_scores).mean(), 'class_report':[classification_reports]})

    return res_avg_metrics, test_dfs


In [ ]:
df = pd.read_csv('df_file.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'df_file.csv'

## Preprocessing

## LogReg/LGBM

In [ ]:
# инициализация моделей для обучения
log_reg = LogisticRegression()
lgbm = LGBMClassifier(n_estimators=150, max_depth=3, learning_rate=0.1, min_child_samples=1)

# инициализация векторайзера
tf_idf_vectorizer_multi = TfidfVectorizer(max_features=20000, ngram_range=(1,3), max_df=0.98)

corpus = list(df['Text'])
stem_corpus = [preprocessing(corpus[i], word_tokenize, stop_words, del_sw=True, lemm=False, stem=True) for i in tqdm(range(len(corpus)))]

detoken_docs = detokenization(stem_corpus)
tf_idf_vectors = tf_idf_vectorizer_multi.fit_transform(detoken_docs)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

# кросс-валидация
logReg_metrics_multi, test_dfs = my_cross_validate(df, log_reg, skf, tf_idf_vectors, target_col_name='Label')
lgbm_metrics_multi, test_dfs = my_cross_validate(df, lgbm, skf, tf_idf_vectors, target_col_name='Label')

# получение результирующих метрик
clear_tags = np.append(df['Label'].unique(), ['macro avg', 'weighted avg'])
metrics = ['precision', 'recall', 'f1-score', 'support']

a = [[[i[tag][metric] for i in logReg_metrics_multi['class_report'][0]] for tag in clear_tags] for metric in metrics]
b = [[np.mean(i) for i in j] for j in a]

c = [[[i[tag][metric] for i in lgbm_metrics_multi['class_report'][0]] for tag in clear_tags] for metric in metrics]
d = [[np.mean(i) for i in j] for j in c]

lr_metrics = pd.DataFrame({'tag':clear_tags, 'precision':b[0], 'recall':b[1], 'f1':b[2], 'support':b[3]})
lgbm_metrics = pd.DataFrame({'tag':clear_tags, 'precision':d[0], 'recall':d[1], 'f1':d[2], 'support':d[3]})





  0%|          | 0/2225 [00:00<?, ?it/s]


  1%|▏         | 30/2225 [00:00<00:07, 295.35it/s]


  3%|▎         | 63/2225 [00:00<00:06, 313.26it/s]


  4%|▍         | 95/2225 [00:00<00:07, 289.09it/s]


  6%|▌         | 125/2225 [00:00<00:07, 288.51it/s]


  7%|▋         | 157/2225 [00:00<00:06, 296.56it/s]


  8%|▊         | 188/2225 [00:00<00:06, 300.89it/s]


 10%|▉         | 222/2225 [00:00<00:06, 310.67it/s]


 12%|█▏        | 257/2225 [00:00<00:06, 320.70it/s]


 13%|█▎        | 290/2225 [00:00<00:06, 319.25it/s]


 14%|█▍        | 322/2225 [00:01<00:06, 305.79it/s]


 16%|█▌        | 355/2225 [00:01<00:06, 310.16it/s]


 17%|█▋        | 387/2225 [00:01<00:06, 299.24it/s]


 19%|█▉        | 418/2225 [00:01<00:06, 285.57it/s]


 21%|██        | 464/2225 [00:01<00:05, 332.98it/s]


 23%|██▎       | 502/2225 [00:01<00:04, 346.23it/s]


 25%|██▍       | 549/2225 [00:01<00:04, 378.91it/s]


 27%|██▋       | 593/2225 [00:01<00:04, 393.90it/s]


 28%|██▊       | 633/2225 [00:01<00:04,

Fold в„–: 0
Label
1    409
4    408
0    333
2    321
3    309
Name: count, dtype: int64


  0%|          | 0/2225 [02:29<?, ?it/s]



1it [00:02,  2.83s/it]

Precision: 0.986603375527426
Recall: 0.9857773109243698
F1: 0.9861272355434064

Fold в„–: 1
Label
1    409
4    408
0    333
2    321
3    309
Name: count, dtype: int64





2it [00:04,  2.25s/it]

...

Precision: 0.9685772357723577
Recall: 0.9647380963253133
F1: 0.9663533945729068

Fold в„–: 3
Label
1    409
4    408
0    334
2    320
3    309
Name: count, dtype: int64
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.600787 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 126072
[LightGBM] [Info] Number of data points in the train set: 1780, number of used features: 19863
[LightGBM] [Info] Start training from score -1.673228
[LightGBM] [Info] Start training from score -1.470653
[LightGBM] [Info] Start training from score -1.716048
[LightGBM] [Info] Start training from score -1.751027
[LightGBM] [Info] Start training from score -1.473101
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

4it [01:27, 22.06s/it]

Precision: 0.9587443730425746
Recall: 0.9570619063413771
F1: 0.9574887274054585

Fold в„–: 4
Label
1    409
4    408
0    334
2    321
3    308
Name: count, dtype: int64
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.573125 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 126586
[LightGBM] [Info] Number of data points in the train set: 1780, number of used features: 19896
[LightGBM] [Info] Start training from score -1.673228
[LightGBM] [Info] Start training from score -1.470653
[LightGBM] [Info] Start training from score -1.712928
[LightGBM] [Info] Start training from score -1.754269
[LightGBM] [Info] Start training from score -1.473101
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

5it [01:51, 22.22s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Precision: 0.9549297400805384
Recall: 0.9553164694979012
F1: 0.9545814546662967



In [ ]:
lr_metrics

,tag,precision,recall,f1,support
0,0,0.978758,0.966351,0.972283,83.4
1,1,0.988460,0.998039,0.993199,102.2
2,2,0.977435,0.970123,0.973741,80.2
3,3,0.981744,0.974059,0.977775,77.2
4,4,0.963663,0.974510,0.968790,102.0
5,macro avg,0.978012,0.976617,0.977158,445.0
6,weighted avg,0.977800,0.977528,0.977505,445.0


In [ ]:
lgbm_metrics

,tag,precision,recall,f1,support
0,0,0.945699,0.944980,0.944803,83.4
1,1,0.979011,0.990215,0.984503,102.2
2,2,0.962703,0.962562,0.962589,80.2
3,3,0.975996,0.948185,0.961609,77.2
4,4,0.942686,0.950980,0.946353,102.0
5,macro avg,0.961219,0.959384,0.959971,445.0
6,weighted avg,0.960998,0.960449,0.960397,445.0


## CNN

In [ ]:
max_features = 5000
max_len = 2000

train_df, test_df = train_test_split(df, test_size=0.3)

# токенезация и векторизация
keras_tokenizer = Tokenizer(num_words=max_features)
keras_tokenizer.fit_on_texts(df['Text'])

# деление на части
sequence_train = keras_tokenizer.texts_to_sequences(train_df['Text'])
sequence_test = keras_tokenizer.texts_to_sequences(test_df['Text'])

# padding последовательностей
X_train = keras.utils.pad_sequences(sequence_train, max_len, padding='post')
X_test = keras.utils.pad_sequences(sequence_test, max_len, padding='post')

y_train = keras.utils.to_categorical(train_df['Label'])
y_test = keras.utils.to_categorical(test_df['Label'])

cnn_model = keras.Sequential()
cnn_model.add(keras.layers.Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
# cnn_model.add(keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
# cnn_model.add(keras.layers.MaxPool1D(pool_size=3))
cnn_model.add(keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
cnn_model.add(keras.layers.MaxPool1D(pool_size=3))
cnn_model.add(keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu'))
# cnn_model.add(keras.layers.Flatten())
cnn_model.add(keras.layers.GlobalMaxPooling1D())
cnn_model.add(keras.layers.Dropout(0.3))
cnn_model.add(keras.layers.Dense(20, activation='relu'))
cnn_model.add(keras.layers.Dropout(0.3))
cnn_model.add(keras.layers.Dense(len(y_train[0]), activation='softmax'))

cnn_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.fit(X_train, y_train, batch_size=32, epochs=20)

cnn_preds = cnn_model.predict(X_test)
cnn_preds = [np.argmax(pred) for pred in cnn_preds]
y_test = [np.argmax(i) for i in y_test]


Epoch 1/20
49/49 [==============================] - 27s 509ms/step - loss: 1.6090 - accuracy: 0.2222
Epoch 2/20
49/49 [==============================] - 26s 522ms/step - loss: 1.5720 - accuracy: 0.3019
Epoch 3/20
49/49 [==============================] - 26s 532ms/step - loss: 1.2900 - accuracy: 0.4701
Epoch 4/20
49/49 [==============================] - 26s 530ms/step - loss: 0.8885 - accuracy: 0.6095
Epoch 5/20
49/49 [==============================] - 26s 537ms/step - loss: 0.5721 - accuracy: 0.7771
Epoch 6/20
49/49 [==============================] - 26s 523ms/step - loss: 0.3381 - accuracy: 0.8992
Epoch 7/20
49/49 [==============================] - 26s 524ms/step - loss: 0.2351 - accuracy: 0.9306
Epoch 8/20
49/49 [==============================] - 26s 531ms/step - loss: 0.1659 - accuracy: 0.9441
Epoch 9/20
49/49 [==============================] - 26s 540ms/step - loss: 0.1113 - accuracy: 0.9634
Epoch 10/20
49/49 [==============================] - 26s 536ms/step - loss: 0.1019 - accura

KeyboardInterrupt: 

## RNN

### Vectorization

In [ ]:
encoder = keras.layers.TextVectorization(max_tokens=10000)
encoder.adapt(df['Text'])

# Extracting the vocabulary from the TextVectorization layer.
vocabulary = np.array(encoder.get_vocabulary())

# Encoding a test example and decoding it back.
example = df.loc[0, 'Text']
original_text = example
encoded_text = encoder(original_text).numpy()
decoded_text = ' '.join(vocabulary[encoded_text])


In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,  return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(np.unique(df['Label'])), activation='softmax')
])

# Summary of the model
model.summary()

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, None, 64)          640000    
                                                                 
 bidirectional_40 (Bidirect  (None, None, 128)         66048     
 ional)                                                          
                                                                 
 bidirectional_41 (Bidirect  (None, 64)                41216     
 ional)                                                          
                                                                 
 dense_48 (Dense)            (None, 64)                4160      
                                                                 
 dense_49 (Dense)            (None, 5)                 325       
                                                                 
Total params: 751749 (2.87 MB)
Trainable params: 7517

In [ ]:
# делим на train и test
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['Label'])

# токенизируем выборки
x_train = [encoder(doc).numpy() for doc in train_df['Text']]
x_test = [encoder(doc).numpy() for doc in test_df['Text']]

# паддим полученные вектора
x_train = pad_sequences(x_train, maxlen=500)
x_test = pad_sequences(x_test, maxlen=500)

y_train = train_df['Label']
y_test = test_df['Label']

In [ ]:
# обучение модели
callback = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.1)
model.fit(x=x_train, y=y_train, epochs=10, callbacks=[callback])

Epoch 1/10
49/49 [==============================] - 84s 1s/step - loss: 1.5303 - accuracy: 0.2755
Epoch 2/10
49/49 [==============================] - 63s 1s/step - loss: 0.7864 - accuracy: 0.7097
Epoch 3/10
49/49 [==============================] - 65s 1s/step - loss: 0.4668 - accuracy: 0.8311
Epoch 4/10
49/49 [==============================] - 63s 1s/step - loss: 0.1854 - accuracy: 0.9550
Epoch 5/10
49/49 [==============================] - 64s 1s/step - loss: 0.0690 - accuracy: 0.9820
Epoch 6/10
49/49 [==============================] - 64s 1s/step - loss: 0.0493 - accuracy: 0.9884


In [ ]:
# предсказания модели
preds = model.predict(x=x_test)

21/21 [==============================] - 14s 374ms/step


In [ ]:
sklearn.metrics.classification_report(y_true=y_test, y_pred=preds)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

## Transformets (BERT)

In [ ]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

encoded_text = tokenizer("The movie was not good")
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(encoded_text,len(encoded_text.input_ids))
print(tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 1996, 3185, 2001, 2025, 2204, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]} 7
['[CLS]', 'the', 'movie', 'was', 'not', 'good', '[SEP]']


In [ ]:
model = AutoModel.from_pretrained(model_ckpt).to(device)